# Markov Chains

## About

In probability theory and statistics, a **Markov chain** is a random process describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event.

[Wiki - Markov Chain](https://en.wikipedia.org/wiki/Markov_chain)

A simple boolean example is if you lost the lottery 10 times in the past, you can resonably say that you will not win the lottery next time you gamble.

## First-Order Markov Chain

A first-order Markov Chain uses the current state, and learned probabilites of transitions, to predict the next state.

-  **States**: A set of possible states.
-  **Transition Probabilities**: probabilities that define the chance of moving from one state to another. The probabilities in each row must add up to 100%.

e.g.

| From \ To | Sunny | Cloudy | Rainy |
|:----------|:------|:-------|:------|
| **Sunny** | 0.7   | 0.2    | 0.1   |
| **Cloudy**| 0.4   | 0.4    | 0.2   |
| **Rainy** | 0.3   | 0.5    | 0.2   |

If it's Sunny today, there's a 70% chance it will be Sunny tomorrow. 

### 1st order example

In [1]:
import numpy as np
import pandas as pd

states = ['Sunny', 'Cloudy', 'Rainy']

# the transition matrix
transition_matrix = np.array([
    [0.7, 0.2, 0.1], # From Sunny
    [0.4, 0.4, 0.2], # From Cloudy
    [0.3, 0.5, 0.2]  # From Rainy
])

transition_df = pd.DataFrame(transition_matrix, index=states, columns=states)
print("Transition Probabilities:")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(transition_df)

Transition Probabilities:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        Sunny  Cloudy  Rainy
Sunny     0.7     0.2    0.1
Cloudy    0.4     0.4    0.2
Rainy     0.3     0.5    0.2


### Markov Chain, Weather Prediction

1. Get the transistion probabilities for the current state.
2. Use the probabilities for the current state to randomly predict the next state.
3. Update the current state and repeat.

In [2]:
import random

def weighted_choice(states, probabilities):
    r = random.random()
    cumulative = 0.0
    for i in range(len(states)):
        cumulative += probabilities[i]
        if r < cumulative:
            return states[i]
    return states[-1]


def predict_weather( start_state, steps):
    
    current_state = start_state
    forecast = [current_state]
    
    for i in range(steps - 1):
        current_index = states.index(current_state)
        
        probabilities = transition_matrix[current_index]
        
        # Get next based on the probabilities
        next_state = weighted_choice(states, probabilities)
        
        forecast.append(next_state)
        current_state = next_state
        
    return forecast

In [3]:
print(" -> ".join( predict_weather('Sunny', 14) ) )

Sunny -> Cloudy -> Sunny -> Sunny -> Cloudy -> Rainy -> Cloudy -> Cloudy -> Rainy -> Rainy -> Rainy -> Cloudy -> Rainy -> Sunny


## Limitations
As a 1st order Markov Chain lacks memory any historical patterns will not appear in the predition.

## Second-Order Markov Chain
A second-order chain considers the previous *two* states.
- **State**: The state against which there are stored transition probabilities is now a pair of distinct states.
-  **Transition Probabilities**: probabilities that define the chance of moving from one state to another. The probabilities in each row must add up to 100%.

In [11]:
input_data = "The cat sat on the mat. The dog sat on the log. The cat chased the dog."

In [12]:
def build_second_order_model(text):
    words = text.split()
    model = {}

    for i in range(len(words) - 2):
        current_pair = (words[i], words[i+1])
        next_word = words[i+2]

        # Initialize dictionary for frequency counts
        if current_pair not in model:
            model[current_pair] = {}

        # Increase frequency of the next_word
        model[current_pair][next_word] = model[current_pair].get(next_word, 0) + 1

    return model

second_order_model = build_second_order_model(input_data)

for pair, next_words in second_order_model.items():
    print(f"{pair} -> {next_words}")

('The', 'cat') -> {'sat': 1, 'chased': 1}
('cat', 'sat') -> {'on': 1}
('sat', 'on') -> {'the': 2}
('on', 'the') -> {'mat.': 1, 'log.': 1}
('the', 'mat.') -> {'The': 1}
('mat.', 'The') -> {'dog': 1}
('The', 'dog') -> {'sat': 1}
('dog', 'sat') -> {'on': 1}
('the', 'log.') -> {'The': 1}
('log.', 'The') -> {'cat': 1}
('cat', 'chased') -> {'the': 1}
('chased', 'the') -> {'dog.': 1}


In [8]:
def generate_text_second_order(model, length):
    # Start with a random pair from the model
    current_pair = random.choice(list(model.keys()))
    text = list(current_pair)

    for _ in range(length - 2):
        if current_pair not in model:
            break

        # Get dictionary: next_word → count
        freq_dict = model[current_pair]

        # Weighted random choice
        words = list(freq_dict.keys())
        weights = list(freq_dict.values())
        total = sum(weights)
        probabilities = [w / total for w in weights]

        next_word = weighted_choice(words, probabilities)

        text.append(next_word)

        # Update pair
        current_pair = (current_pair[1], next_word)

    return " ".join(text)

In [13]:
output = generate_text_second_order(second_order_model, 20)
print(output)

sat on the mat. The dog sat on the log. The cat chased the dog.
